USE YOUR OWN SEC API KEYS LOL

In [ ]:
import requests
import json
import pandas as pd

fdi_key = {
    # Request headers of fund daily info
    'Ocp-Apim-Subscription-Key': 'afbbb604d2814b59b0e5fa6ff3717e16'
}

ffs_key = {
    # Request headers of fund fact sheet
    'Ocp-Apim-Subscription-Key': '16c86c3eb3cb4fee8f54932b92630aef'
}

NAV FURTHEST POSSIBLE, Date (format as of)
unique_id บลจ.SCB
2018-2019

In [ ]:
unique_id = "C0000000239"
fundfactsheet = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/amc/{unique_id}', headers = ffs_key)
# The total of mutual fund which SCB is occupied

In [ ]:
proj_list = json.loads(fundfactsheet.text)

In [ ]:
list_unique_id = []
for x in proj_list :
    list_unique_id.append(x['proj_id'])

In [ ]:
list_rmf = []
list_ssf = []
for x in list_unique_id :
    fundfactsheet = requests.get(f'https://api.sec.or.th/FundFactsheet/fund/{x}/specification', headers = ffs_key)
    print(fundfactsheet.status_code)
    if fundfactsheet.status_code == 200:
        for j in json.loads(fundfactsheet.text):
            if j['spec_code'] == 'SSF' :
                list_ssf.append(x)
            elif j['spec_code'] == 'RMF':
                list_rmf.append(x)

In [ ]:
print(len(list_ssf))
list_ssf

Date Time

In [ ]:
from datetime import date, timedelta
from datetime import datetime

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [ ]:
end_date = datetime(2022, 12, 31)

In [ ]:
import csv
header = ['nav_date',
'net_asset',
'last_val',
'sell_price',
'buy_price',
'sell_swap_price',
'buy_swap_price']
for ssf_id in list_ssf:
    year =  ssf_id.rsplit("_", 1) #Split _ to dictate years in Thai Calendar, TH year - 543 = A.D., B Year
    start_year = (int)(year[1])-543 #At index 1, in list String year is Year in TH, then we subtract by 543
    start_date = datetime(start_year,1,1)
    filename = f'ssf_nav_data/'+ssf_id+'_SSF.csv'
    with open(filename, 'w',newline = '') as f_out:    
        f_out = csv.writer(f_out,quoting=csv.QUOTE_ALL)
        f_out.writerow(header)
        for date in daterange(start_date,datetime(2022, 12, 31)):
                res_FDI = requests.get(f'https://api.sec.or.th/FundDailyInfo/{ssf_id}/dailynav/{date.strftime("%Y-%m-%d")}', headers = fdi_key)
                print(res_FDI)
                print(date)
                if(res_FDI.status_code == 200):
                    res_json = json.loads(res_FDI.text)
                    f_out.writerow([res_json['nav_date'],res_json['net_asset'],res_json['last_val'],
                                 res_json['amc_info'][0]['sell_price'],
                                 res_json['amc_info'][0]['buy_price'],
                                 res_json['amc_info'][0]['sell_swap_price'],
                                 res_json['amc_info'][0]['buy_swap_price']])
         
